# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [6]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [7]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    #TODO 1: Add your code here
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    
    column_mapping = {
        "Title" : "Title",
        "Year" : "Year",
        "Synopsis" : "Synopsis", 
        "Review" : "Review"
    }
    
    df_eng.columns = column_mapping.values()
    df_sp.columns = column_mapping.values()
    df_fr.columns = column_mapping.values()
    
    df_eng["Original Language"] = "English"
    df_sp["Original Language"] = "Spanish"
    df_fr["Original Language"] = "French"
    
    df = pd.concat([df_eng, df_sp, df_fr], ignore_index = True)
    
    return df

df = preprocess_data()

In [8]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",English
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English
29,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",French
24,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",French
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
21,Intouchables,2011,Ce film raconte l'histoire de l'amitié improba...,"""Intouchables est un film incroyablement touch...",French
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
28,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",French
14,Amores perros,2000,Tres historias se entrelazan en esta película ...,"""Amores perros es una película intensa y conmo...",Spanish
22,Amélie,2001,Cette comédie romantique raconte l'histoire d'...,"""Amélie est un film absolument charmant qui vo...",French


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [9]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)
# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    Function to translate a text using a model and tokenizer.
    """
    # encode the text using the tokenizer
    inputs = tokenizer.encode(text, return_tensors="pt")
    # generate the translation using the model
    outputs = model.generate(inputs)
    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded


/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [10]:
# TODO 4: Update the code below
df = preprocess_data()

# Filter reviews and synopses in French and translate to English
fr_reviews = df[df['Original Language'] == 'French']['Review']
fr_reviews_en = fr_reviews.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

fr_synopsis = df[df['Original Language'] == 'French']['Synopsis']
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# Filter reviews and synopses in Spanish and translate to English
es_reviews = df[df['Original Language'] == 'Spanish']['Review']
es_reviews_en = es_reviews.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

es_synopsis = df[df['Original Language'] == 'Spanish']['Synopsis']
es_synopsis_en = es_synopsis.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

# Update dataframe with translated text
# Update French reviews and synopses
df.loc[df['Original Language'] == 'French', 'Review'] = fr_reviews_en.values
df.loc[df['Original Language'] == 'French', 'Synopsis'] = fr_synopsis_en.values

# Update Spanish reviews and synopses
df.loc[df['Original Language'] == 'Spanish', 'Review'] = es_reviews_en.values
df.loc[df['Original Language'] == 'Spanish', 'Synopsis'] = es_synopsis_en.values


/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [11]:

df.sample(10)

,Title,Year,Synopsis,Review,Original Language
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English
27,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",French
18,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish
22,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",French
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
20,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",French
11,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",Spanish


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [12]:
# TODO 5: Update the code below
# load sentiment analysis model
from transformers import pipeline
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)


# create sentiment analysis function
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    result = classifier(text)
    label = result[0]["label"]
    score = result[0]["score"]
    return label, score

In [13]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df["Sentiment"] = df["Review"].apply(lambda x: sentiment_classifier(x)[0]["label"])


In [14]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
23,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",French,POSITIVE
18,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish,NEGATIVE
19,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",Spanish,NEGATIVE
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English,NEGATIVE
21,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",French,POSITIVE
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",English,POSITIVE
28,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",French,NEGATIVE
10,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",Spanish,POSITIVE
15,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",Spanish,NEGATIVE
12,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",Spanish,POSITIVE


In [15]:
# export the results to a .csv file
df.to_csv("reviews_with_sentiment.csv", index=False)